In [ ]:
import json
import  pandas as pd

base_dir ="H:\文档\门架收费站研究"
gantry_path = "2023年3~4整合门架"
toll_path = "2023年3~4整合出入口"
gantries = ["4D2806","4D2805","4D2804"]
toll_stations = ["广东市南路站","广东亚运城站"]

from datetime import datetime, timedelta

start_date = datetime(2023, 3, 1, 0, 0)
end_date = datetime(2023, 4, 30, 12, 0)
interval = timedelta(minutes=1)
current_date = start_date
minute_time_list = []
# 遍历一天的时间并格式化为字符串
while current_date <= end_date:  # 假设要遍历的日期是2月10日
    formatted_time = current_date.strftime("%Y-%m-%d %H:%M")
    minute_time_list.append(formatted_time)
    current_date += interval
minute_time_list


In [ ]:
# 读取文本文件内容
import os
gantry_df_list = []
folder_path1 = f"{base_dir}\\{gantry_path}"
for gp in [os.path.join(folder_path1, file_name) for file_name in os.listdir(folder_path1)]:
    with open(gp, 'r',encoding="utf-8") as file:
        data_text = file.read()
    data_json = [json.loads(line) for line in data_text.strip().split('\n')]
    gantry_df_list.append(pd.DataFrame(data_json))
gantry_df0  = pd.concat(gantry_df_list)

gantry_df0.columns = ["pass_id","l"]
gantry_df_list = []
for row in gantry_df0.itertuples():
    new_row = {"pass_id":getattr(row,'pass_id')}
    gantry_l = getattr(row,'l').split(",")
    if len(gantry_l)==3:
        new_row["l1"] = gantry_l[0].replace("[","").replace("\\","").replace("\"","").replace("]","").replace("\"","")
        new_row["l2"] = gantry_l[1].replace("[","").replace("\\","").replace("\"","").replace("]","").replace("\"","")
        new_row["l3"] = gantry_l[2].replace("[","").replace("\\","").replace("\"","").replace("]","").replace("\"","")
    elif len(gantry_l)==2:
        new_row["l1"] = gantry_l[0].replace("[","").replace("\\","").replace("\"","").replace("]","").replace("\"","")
        new_row["l2"] = gantry_l[1].replace("[","").replace("\\","").replace("\"","").replace("]","").replace("\"","")
        new_row["l3"] = ""
    else:
        new_row["l1"] = gantry_l[0].replace("[","").replace("\\","").replace("\"","").replace("]","").replace("\"","")
        new_row["l2"] = ""
        new_row["l3"] = ""
    gantry_df_list.append(new_row)
gantry_df = pd.DataFrame(gantry_df_list)


In [ ]:
gantry_df_list

In [ ]:

gantry_df["l1_time"] = ""
gantry_df["l1_time"] =  gantry_df["l1"].agg(lambda x:x.split("----")[0]  if x!="" else "")
gantry_df["l1_gantry"]= gantry_df["l1"].agg(lambda x:x.split("----")[1]  if x!="" else "")
gantry_df["l2_time"] =  gantry_df["l2"].agg(lambda x:x.split("----")[0]  if x!="" else "")
gantry_df["l2_gantry"]= gantry_df["l2"].agg(lambda x:x.split("----")[1]  if x!="" else "")
gantry_df["l3_time"] =  gantry_df["l3"].agg(lambda x:x.split("----")[0]  if x!="" else "")
gantry_df["l3_gantry"]= gantry_df["l3"].agg(lambda x:x.split("----")[1]  if x!="" else "")
gantry_df["total"]= gantry_df["l1"]+gantry_df["l3"]+gantry_df["l3"]
# gantry_df2 = gantry_df.query("l1_gantry=='4D290A' or l2_gantry=='4D290A' or l3_gantry=='4D290A'")
a_list = gantry_df.query(f"l1_gantry=='{gantries[0]}'")["l1_time"].to_list()+gantry_df.query(f"l2_gantry=='{gantries[0]}'")["l2_time"].to_list()+gantry_df.query(f"l3_gantry=='{gantries[0]}'")["l3_time"].to_list()
a_df = pd.DataFrame({"time":a_list,"value":[x for x in range(len(a_list))]})
grouped  = a_df.groupby("time").count()
sorted_df = grouped.sort_values(by='value', ascending=False)
print(sorted_df)

In [ ]:
gantry_array = gantry_df[["pass_id","l1","l2","l3"]].to_numpy()

new_gantry_df_list = []
for row in gantry_array:
    row_dict = {"pass_id":row[0]}
    for gantry in gantries:
        row_dict["gantry_"+gantry] = "no"
        row_dict["time_"+gantry] = "no"
    for record in row[1:]:
        for gantry in gantries:
            if gantry in record:
                row_dict["gantry_"+gantry]  = gantry
                row_dict["time_"+gantry]  = record.split("----")[0]
    new_gantry_df_list.append(row_dict)
new_gantry_df = pd.DataFrame(new_gantry_df_list)
new_gantry_df

In [ ]:
final_gantry_df = new_gantry_df.query(f"(time_{gantries[0]} >= '{minute_time_list[0]}' and time_{gantries[0]} <='{minute_time_list[-1]}') or (time_{gantries[0]} =='no' and time_{gantries[1]}>= '{minute_time_list[0]}' and time_{gantries[1]} <='{minute_time_list[-1]}') or  (time_{gantries[0]} =='no' and time_{gantries[1]}=='no' and time_{gantries[2]}>= '{minute_time_list[0]}' and time_{gantries[2]} <='{minute_time_list[-1]}')")
final_gantry_df = final_gantry_df.sort_values(by=f'time_{gantries[0]}')
conditions = [f"gantry_{gantries[0]} != 'no'",
              f"gantry_{gantries[0]} == 'no'",
              f"gantry_{gantries[1]} != 'no'",
              f"gantry_{gantries[1]} == 'no'",
              f"gantry_{gantries[2]} != 'no'",
              f"gantry_{gantries[2]} == 'no'"]
con_ABC = final_gantry_df.query(f"{conditions[0]} and {conditions[2]} and {conditions[4]}")
con_AB = final_gantry_df.query(f"{conditions[0]} and {conditions[2]} and {conditions[5]}")
con_A = final_gantry_df.query(f"{conditions[0]} and {conditions[3]} and {conditions[5]}")
con_BC = final_gantry_df.query(f"{conditions[1]} and {conditions[2]} and {conditions[4]}")
con_B = final_gantry_df.query(f"{conditions[1]} and {conditions[2]} and {conditions[5]}")
con_C = final_gantry_df.query(f"{conditions[1]} and {conditions[3]} and {conditions[4]}")
cons = {'C123':con_ABC,'C12':con_AB,'C1':con_A,'C23':con_BC,'C2':con_B,'C3':con_C}
con_ABC

In [ ]:
toll_df_list = []
folder_path2 = f"{base_dir}\\{toll_path}"
for tp in [os.path.join(folder_path2, file_name) for file_name in os.listdir(folder_path2)]:
    with open(tp, 'r',encoding='utf-8') as file:
        data_text2 = file.read()
    data_json2 = [json.loads(line) for line in data_text2.strip().split('\n')]
    toll_df_list.append(pd.DataFrame(data_json2))

toll_df= pd.concat(toll_df_list)

toll_df["en_time"] = toll_df["en_mess"].agg(lambda x: x.split("----")[0])
toll_df["en_station"] = toll_df["en_mess"].agg(lambda x: x.split("----")[1])
toll_df["ex_time"] = toll_df["ex_mess"].agg(lambda x: x.split("----")[0])
toll_df["ex_station"] = toll_df["ex_mess"].agg(lambda x: x.split("----")[1])
for k,v in cons.items():
    cons[k] = pd.merge(left=v,right=toll_df,on="pass_id",how="left").fillna("no")
    cons[k].to_csv(f"res/{k}.csv",index=False)


In [ ]:
# cons['C123_0_c'] = cons['C123'].query(f"ex_station=='{toll_stations[2]}'")
# cons['C123_0_0'] = cons['C123'].query(f"ex_station=='no'")
cons['C12_0_b'] = cons['C12'].query(f"ex_station=='{toll_stations[1]}'")
cons['C1_0_a'] = cons['C123'].query(f"ex_station=='{toll_stations[0]}'")
# cons['C23_a_0'] = cons['C23'].query(f"en_station=='{toll_stations[0]}' and ex_station=='no'")
# cons['C23_a_c'] = cons['C23'].query(f"en_station=='{toll_stations[0]}' and ex_station=='{toll_stations[2]}'")
cons['C2_a_b'] = cons['C23'].query(f"en_station=='{toll_stations[0]}' and ex_station=='{toll_stations[1]}'")
cons['C3_b_0'] = cons['C3'].query(f"en_station=='{toll_stations[1]}'")
# cons['C3_b_c'] = cons['C3'].query(f"en_station=='{toll_stations[1]}' and ex_station=='{toll_stations[2]}'")
cons['C12_0_b']

In [ ]:
import  pyperclip
# pyperclip.copy(str([str(x) for x in cons['C12'].query("en_station=='no' and ex_station=='no'")['pass_id'].to_list()]))
# cons['C123']
# len_calcu = {k:str(len(v.query("en_station=='no' and ex_station=='no'")))+"/"+str(len(v)) for k,v in cons.items()}
# len_calcu
cons["Ca"] = cons["C"]


In [ ]:
# 各个分钟有多少个各个条件的车通过
minute_nums = []
for minute in minute_time_list:
    con_nums = {"time":minute}
    for k,v in cons.items():
        con_nums[k] = len(v.query(f"time_{gantries[int(k[1])-1]}==@minute"))
    minute_nums.append(con_nums)
pd.DataFrame(minute_nums)
import pickle
with open("res/minute_res.pkl","wb") as f:
    pickle.dump(minute_nums,f)